In [1]:
## PYSPARK SESSION 
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import sys
from pyspark.sql import SparkSession

spark = SparkSession.builder \
.master("local") \
.appName("Exercise1") \
.getOrCreate()


In [2]:
## GET DATA (LOCAL)
df = spark.read.csv("/home/jovyan/data/exampleData.csv", header=True, inferSchema=True)
df.show(5,vertical=False) #show 5 values

df.createOrReplaceTempView("weatherConditions")
staticSchema = df.schema

+---+-----------------+--------------+-------------+--------------+---------------+--------------+----+-----+-----+-----------------+------+--------------+----------+---------------+-------------------+
|_c0|         dateTime|indicator_rain|precipitation|indicator_temp|air_temperature|indicator_wetb|wetb|dewpt|vappr|relative_humidity|   msl|indicator_wdsp|wind_speed|indicator_wddir|wind_from_direction|
+---+-----------------+--------------+-------------+--------------+---------------+--------------+----+-----+-----+-----------------+------+--------------+----------+---------------+-------------------+
| 14|01-jan-1990 00:00|             0|          0.3|             0|            9.1|             0| 9.0|  8.9| 11.4|               99|1006.7|             2|         7|              2|                190|
| 15|01-jan-1990 01:00|             0|          0.2|             0|            8.2|             0| 7.4|  6.4|  9.7|               89|1006.7|             2|         5|              2|      

In [ ]:
#LOAD FROM URL

import urllib.request

# Download the file from the URL
url = "https://cli.fusio.net/cli/climate_data/webdata/hly1075.csv"
local_file_path = "/home/jovyan/data/exampleData_url.csv"  # Set your local file path here
urllib.request.urlretrieve(url, local_file_path)

# Read the downloaded file using PySpark
df_url = spark.read.csv(local_file_path, header=True, inferSchema=True)

# Show the DataFrame
#df.show(vertical=False)
df_url.show(20)

# Create a temporary view
df_url.createOrReplaceTempView("weatherConditions")

# Store the schema
staticSchema = df_url.schema


In [3]:
from pyspark.sql import functions as F
df.filter((F.col('air_temperature')<5))
df.show(5)

DataFrame[_c0: int, dateTime: string, indicator_rain: int, precipitation: string, indicator_temp: int, air_temperature: string, indicator_wetb: int, wetb: string, dewpt: string, vappr: string, relative_humidity: string, msl: string, indicator_wdsp: int, wind_speed: string, indicator_wddir: int, wind_from_direction: int]

+---+-----------------+--------------+-------------+--------------+---------------+--------------+----+-----+-----+-----------------+------+--------------+----------+---------------+-------------------+
|_c0|         dateTime|indicator_rain|precipitation|indicator_temp|air_temperature|indicator_wetb|wetb|dewpt|vappr|relative_humidity|   msl|indicator_wdsp|wind_speed|indicator_wddir|wind_from_direction|
+---+-----------------+--------------+-------------+--------------+---------------+--------------+----+-----+-----+-----------------+------+--------------+----------+---------------+-------------------+
| 14|01-jan-1990 00:00|             0|          0.3|             0|            9.1|             0| 9.0|  8.9| 11.4|               99|1006.7|             2|         7|              2|                190|
| 15|01-jan-1990 01:00|             0|          0.2|             0|            8.2|             0| 7.4|  6.4|  9.7|               89|1006.7|             2|         5|              2|      

In [4]:
## spark streaming
streamingDF = spark.readStream\
.schema(staticSchema)\
.option("maxFilesPerTrigger", 3)\
.format("csv")\
.option("header", "true")\
.load("data")

#streamingDF.writeStream.format("console").start()
#streamingDF.show()

streamingDF.isStreaming # if its returns True , we are good to go


True

In [ ]:
## SCRAP FROM JSON 

from pyspark.sql import SparkSession

# Initialize SparkSession
spark = SparkSession.builder \
    .appName("JSON_Data_Read") \
    .getOrCreate()

# Read JSON file into DataFrame
df_json = spark.read.json("jsonData.json")

# Show DataFrame
df_json.show()
jsonSchema = df_json.schema
# Stop SparkSession
#spark.stop()




In [5]:
## FILTER STREAM DATA

filteredDF = streamingDF.filter((F.col('air_temperature')<5))

In [6]:
## Writing Stream to a sink

df = filteredDF.writeStream.format("memory") \
.queryName("weatherQuery") \
.outputMode("append") \
.start()
df.awaitTermination()

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.9.3-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/usr/local/spark/python/lib/py4j-0.10.9.3-src.zip/py4j/clientserver.py", line 475, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/opt/conda/lib/python3.9/socket.py", line 704, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

In [7]:
spark.sql("""
SELECT *
FROM weatherQuery
""")\
.show(5)

+---+--------+--------------+-------------+--------------+---------------+--------------+----+-----+-----+-----------------+---+--------------+----------+---------------+-------------------+
|_c0|dateTime|indicator_rain|precipitation|indicator_temp|air_temperature|indicator_wetb|wetb|dewpt|vappr|relative_humidity|msl|indicator_wdsp|wind_speed|indicator_wddir|wind_from_direction|
+---+--------+--------------+-------------+--------------+---------------+--------------+----+-----+-----+-----------------+---+--------------+----------+---------------+-------------------+
+---+--------+--------------+-------------+--------------+---------------+--------------+----+-----+-----+-----------------+---+--------------+----------+---------------+-------------------+

